In [97]:
import pandas as pd
import re

# File reader

In [98]:
data = pd.read_csv('representative_logFile.csv', decimal=',')

data.head()

,Start of experiment,20220729_1024,Unnamed: 2,Unnamed: 3
0,Source file name: shape_dotsB_C.txt,NaN,NaN,NaN
1,Screen resolution h: 1920,NaN,NaN,NaN
2,Screen resolution v: 1080,NaN,NaN,NaN
3,Diagonal inch: 31,5,NaN,NaN
4,Horizontal angle: 44,64,NaN,NaN


# Read the patient name and save it

In [99]:
patient = (data[data['Start of experiment'].astype('str').str.contains('Patient name')]).iloc[0,0]

p_name = patient.split(': ')
p_name = p_name[1]

p_name

'Patient_Test'

# Get the row index for:
- responses block
- params and thresholds block

In [100]:
r_block_start = (data.index[
    data['Start of experiment'] == 'Starting experiment number 1...'
].tolist())[0] +1

r_block_end = (data.index[
    data['Start of experiment'] == 'Field_Scale21 value for positive stimuli: 1.0'
].tolist())[0]

r_block = data.iloc[r_block_start: r_block_end,:]
r_block

,Start of experiment,20220729_1024,Unnamed: 2,Unnamed: 3
33,Event time;Event type;Trial;Duration;Selection...,NaN,NaN,NaN
34,11,284;Screen change,NaN,NaN
35,11,838;Trial display,NaN,NaN
36,12,789;Right chosen,NaN,NaN
37,13,148;Trial summary;1;1,31;r;r;1;;;0;0,76
...,...,...,...,...
153,74,664;Trial summary;30;2,024;l;r;0;;;1;0,05797104017732946
154,74,666;Screen change,NaN,NaN
155,75,184;Trial display,NaN,NaN
156,77,207;Left chosen,NaN,NaN


In [101]:
thr_block_start = (data.index[
    data['Start of experiment'] == 'Trial number;PARAM;THRESHOLD;POSITIVE;'    
].tolist())[0]

thr_block = data.iloc[thr_block_start:,:]
thr_block.head()

,Start of experiment,20220729_1024,Unnamed: 2,Unnamed: 3
160,Trial number;PARAM;THRESHOLD;POSITIVE;,NaN,NaN,NaN
161,1;0,62;0,9453125905078741;1,0;
162,2;0,6545454545454545;0,9453125905078741;1,0;
163,3;0,6859504132231405;0,9453125905078741;1,0;
164,4;0,7145003756574005;0,9453125905078741;1,0;


# Set the response block and the params and threshold block as separate DataFrames

In [102]:
responses = pd.read_csv(
    'representative_logFile.csv',
    sep=';',
    header=1,
    skiprows= r_block_start,
    nrows = (r_block_end - r_block_start) -1,
    decimal=',',
    engine='python'
)
responses

,Event time,Event type,Trial,Duration,Selection,Correct,Success,Experimentator,External Stimuli,Reversal,"Staircase level,,,"
0,11.284,"Screen change,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
1,11.838,"Trial display,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
2,12.789,"Right chosen,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
3,13.148,Trial summary,1.0,1.310,r,r,1.0,NaN,NaN,0.0,0.760000
4,13.150,"Screen change,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
119,74.664,Trial summary,30.0,2.024,l,r,0.0,NaN,NaN,1.0,0.057971
120,74.666,"Screen change,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
121,75.184,"Trial display,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
122,77.207,"Left chosen,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN


In [103]:
thresholds = pd.read_csv(
    'representative_logFile.csv',
    sep=';',
    skiprows= thr_block_start+1,
    decimal=',',
    engine='python'
)

thresholds = (thresholds.drop([',,,', 'POSITIVE'],axis=1))[:-1]

# Eventually we are left with two new  dataframes:
- responses: store the response block with times and accuracies
- thresholds: store the params value and the threshold

In [105]:
responses

,Event time,Event type,Trial,Duration,Selection,Correct,Success,Experimentator,External Stimuli,Reversal,"Staircase level,,,"
0,11.284,"Screen change,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
1,11.838,"Trial display,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
2,12.789,"Right chosen,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
3,13.148,Trial summary,1.0,1.310,r,r,1.0,NaN,NaN,0.0,0.760000
4,13.150,"Screen change,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
119,74.664,Trial summary,30.0,2.024,l,r,0.0,NaN,NaN,1.0,0.057971
120,74.666,"Screen change,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
121,75.184,"Trial display,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
122,77.207,"Left chosen,,",NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN


In [106]:
thresholds

,Trial number,PARAM,THRESHOLD
0,1.0,0.62,0.945313
1,2.0,0.6545454545454545,0.945313
2,3.0,0.6859504132231405,0.945313
3,4.0,0.7145003756574005,0.945313
4,5.0,0.7404548869612733,0.945313
5,6.0,0.7640498972375211,0.945313
6,7.0,0.7854999065795647,0.945313
7,8.0,0.8049999150723316,0.945313
8,9.0,0.8227271955203015,0.945313
9,10.0,0.8388429050184558,0.945313
